<a href="https://colab.research.google.com/github/tercerolopezalexisuriel55/ComputationalMethods/blob/master/examenfinal2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Apoyo computacional del examen final 2 
Alexis Uriel Tercero Lopez

'''
#-------------------------------------------------------------------------------
#pregunta c Resuelva el sistema de ecuaciones no lineales
#-------------------------------------------------------------------------------
from math import *
xn = [2,2.5,3,3.5,4,4.5]
yn = [0.57193, 0.63864, 0.71312, 0.79630, 0.88917, 0.99288]

xto2 = 0
ysum = 0
xysum = 0
xsum = 0
m = len(xn)
for i in range(m):
  xto2 = xto2 + xn[i]**2
  ysum = ysum + yn[i]
  xysum = xysum + xn[i]*yn[i]
  xsum = xsum + xn[i]

log2b = (xto2*ysum - xysum*xsum)/(m*xto2 - xsum**2) #8.1
a = (m*xysum - xsum*ysum)/(m*xto2 -  xsum**2)   #8.2
a = round(a,6)
log2b = round(log2b,6)
print('a = ',a)
print('log2b = ',log2b)
b = 2**log2b
b = round(b,6)
print('b = ',b)

def fc(x):
  return b*2**(a*x)

#-------------------------------------------------------------------------------
import numpy as np

def NCubicSplines(x,a,r=2):
  '''Trazador cúbico natural - Algoritmo 3.4 (P:146)
  S(x)=Sj(x)=aj+bj(x-xj)+cj(x-xj)**2+dj(x-xj)**3
  Parametros
    x: numpy.array - puntos muestra.
    a: numpy.array - mapeo de puntos muestra fx.
    r: int - número de decimalies para redondear.
  Retorna
    Q: numpy.array shape = (n,4)
      matriz de incognitas a,b,c,d de cada trazador cubico Sj(x) donde
      Q[j] = [aj,bj,cj,dj]
  '''
  h = np.diff(x) #paso1
  #paso 2
  n = len(x) #nnumero de elmentos
  alpha = np.zeros(n)#[]#alphai
  for i in range(1,n-1):
    alpha[i] = (3/h[i])*(a[i+1]-a[i]) - (3/h[i-1])*(a[i]-a[i-1])
  #paso 3
  l = np.zeros(n)
  mu = np.zeros(n)
  z = np.zeros(n)
  l[0]=1
  mu[0]=0
  z[0]=0
  #paso 4
  for i in range(1,n-1):
    l[i] = 2*(x[i+1]-x[i-1]) - h[i-1]*mu[i-1]
    mu[i] = h[i]/l[i]
    z[i] = (alpha[i]-h[i-1]*z[i-1])/l[i]
  #paso 5
  l[-1] = 1
  z[-1] = 0
  c = np.zeros(n)
  #paso 6
  b = np.zeros(n)
  d = np.zeros(n)
  for j in range(n-2,-1,-1):
    c[j] = z[j] - mu[j]*c[j+1]
    b[j] = (a[j+1]-a[j])/h[j] - h[j]*(c[j+1]+2*c[j])/3
    d[j] = (c[j+1]-c[j])/(3*h[j])
  Q = np.array([np.around(a,r),
                   np.around(b,r),
                   np.around(c,r),
                   np.around(d,r)])
  return Q.transpose()

def InterpolanteCS(xn,yn,r=2):
  def S(x):
    '''S(x)=Sj(x)=aj+bj(x-xj)+cj(x-xj)**2+dj(x-xj)**3'''
    Q = NCubicSplines(xn,yn,r)
    n = len(xn)
    for i in range(n-1): #S_i(x)
      if (x >= xn[i]) and (x <= xn[i+1]):
        a = Q[i][0]
        b = Q[i][1]
        c = Q[i][2]
        d = Q[i][3]
        return a + b*(x-xn[i]) + c*(x-xn[i])**2 + d*(x-xn[i])**3
  return S
#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
#modulo integracion de romberg RombergInt.py 
#import RombergInt as RI

def romberg( f, a, b, n ):
  """Estima la integral de f(x) en [a,b] usando la Integracion de Romberg.
  uso:
      r = romberg( f, a, b, n )

  entrada:
      f       - funcion a integrar,
      [a, b]  - intervalo de integracion,
      n       - para R_{n,n} nivel de recursion

  salida:
      numpy float array - tabla de valores
  """
  #paso1
  r = np.array( [[0] * (n+1)] * (n+1), float )
  h = b - a
  #paso2
  r[0,0] = 0.5 * h * ( f( a ) + f( b ) )
  #paso 3
  powerOf2 = 1
  for i in range( 1, n + 1 ):#paso8
    #Paso 4 (aproximacion metodo del trapécio)
    h = 0.5 * h #paso7
    sum = 0.0
    powerOf2 = 2 * powerOf2
    for k in range( 1, powerOf2, 2 ):
      sum = sum + f( a + k * h )
  
    r[i,0] = 0.5 * r[i-1,0] + sum * h

    #paso 5
    #uso de la ecuacion 4.35 (Extrapolacion de Richardson)
    powerOf4 = 1
    for j in range( 1, i + 1 ):
      powerOf4 = 4 * powerOf4
      #paso 6
      r[i,j] = r[i,j-1] + ( r[i,j-1] - r[i-1,j-1] ) / ( powerOf4 - 1 )
  #paso9
  return r

#programa examen final 2 Aleis Tercero
import matplotlib.pyplot as plt
#-------------------------------------------------------------------------------
#pregunta e hallar el trazador cúbico de los datos dados.
#-------------------------------------------------------------------------------
#Datos del examen
xn = [2,2.5,3,3.5,4,4.5]
fn = [0.57193, 0.63864, 0.71312, 0.79630, 0.88917, 0.99288]
S = InterpolanteCS(xn,fn) #Este es el trazador cubico

xmin, xmax = 1, 5
num = 400
xi = np.linspace( xmin, xmax, num )
sn = [S(x) for x in xi]

#mostrar datos------------------------------------------------------------------
plt.figure( figsize=(12,6) )
plt.plot(xn,fn,'o', color="red", label="datos")
plt.plot(xi, sn, color="green", linewidth=2, label="Interpolacion" )
plt.legend( loc = 'lower left' )
plt.grid()
plt.xlabel( "x" )
plt.ylabel( "y" )
plt.ylim( (0,2) )
plt.xlim( (1,5) )
plt.title( "Interpolación de trazadores cúbicos\nAlexis Tercero" )
plt.show()
#-------------------------------------------------------------------------------
#Pregunta f
'''
Integre a la función obtenida en c) 
en el intervalo de 2 a 4.5.  Integre el trazador cúbico en el mismo intervalo. 
Compare ambos resultados con  R3,3.
'''
#PROGRAMA 
from math import *
n = 3
a = 2
b = 4.5
f = S #Este es el trazador cubico
T = romberg(f, a, b, n)
T = T.round(6)
print(T)
print('Romberg de trazador cubico R_3,3 = ',T[-1][-1])
print()

T1 = romberg(fc, a, b, n)
T1 = T.round(6)
print(T1)
print('Romberg de funcion resultante R_3,3 = ',T1[-1][-1])